## 预处理behavior_log数据集

检查数据格式，以及对数据进行简单的统计运算(不同用户不同行为的次数)


In [1]:
import os
# 配置spark driver和pyspark运行时，所使用的python解释器路径
PYSPARK_PYTHON = "/root/miniconda2/envs/bigdata/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

In [2]:
# spark配置信息
from pyspark import SparkConf
from pyspark.sql import SparkSession

SPARK_APP_NAME = "preprocessingBehaviorLog"
SPARK_URL = "spark://192.168.19.137:7077"

conf = SparkConf()    # 创建spark config对象
config = (
    ("spark.app.name", SPARK_APP_NAME),    # 设置启动的spark的app名称，没有提供，将随机产生一个名称
    ("spark.executor.memory", "2g"),    # 设置该app启动时占用的内存用量，默认1g
    ("spark.master", SPARK_URL),    # spark master的地址
    ("spark.executor.cores", "2"),    # 设置spark executor使用的CPU核心数
    # 以下三项配置，可以控制执行器数量
    # ("spark.dynamicAllocation.enabled", True),
    # ("spark.dynamicAllocation.initialExecutors", 1),    # 1个执行器
    # ("spark.shuffle.service.enabled", True)
    # ('spark.sql.pivotMaxValues', '99999'),  # 当需要pivot DF，且值很多时，需要修改，默认是10000
)
# 查看更详细配置及说明：https://spark.apache.org/docs/latest/configuration.html

conf.setAll(config)

# 利用config对象，创建spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
!hadoop fs -ls /workspace/3.rs_project/project1/dataset

Found 4 items
drwxr-xr-x   - root supergroup           0 2019-03-26 20:22 /workspace/3.rs_project/project1/dataset/ad_feature.csv
-rw-r--r--   1 root supergroup 23728773580 2019-03-26 20:35 /workspace/3.rs_project/project1/dataset/behavior_log.csv
drwxr-xr-x   - root supergroup           0 2019-03-26 20:36 /workspace/3.rs_project/project1/dataset/raw_sample.csv
drwxr-xr-x   - root supergroup           0 2019-03-26 20:36 /workspace/3.rs_project/project1/dataset/user_profile.csv


#### [spark.read.csv](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=read%20csv#pyspark.sql.DataFrameReader.csv)

In [5]:
# 从hdfs加载CSV文件为DataFrame
df = spark.read.csv("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/dataset/behavior_log.csv", header=True)
df.show()    # 查看dataframe，默认显示前20条
# 大致查看一下数据类型
df.printSchema()    # 打印当前dataframe的结构

+------+----------+----+-----+------+
|  user|time_stamp|btag| cate| brand|
+------+----------+----+-----+------+
|558157|1493741625|  pv| 6250| 91286|
|558157|1493741626|  pv| 6250| 91286|
|558157|1493741627|  pv| 6250| 91286|
|728690|1493776998|  pv|11800| 62353|
|332634|1493809895|  pv| 1101|365477|
|857237|1493816945|  pv| 1043|110616|
|619381|1493774638|  pv|  385|428950|
|467042|1493772641|  pv| 8237|301299|
|467042|1493772644|  pv| 8237|301299|
|991528|1493780710|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780714|  pv| 7270|274795|
|991528|1493780765|  pv| 7270|274795|
|991528|1493780714|  pv| 7270|274795|
|991528|1493780765|  pv| 7270|274795|
|991528|1493780764|  pv| 7270|274795|
|991528|1493780633|  pv| 7270|274795|
|991528|1493780764|  pv| 7270|274795|
+------+----------+----+-----+------+
only showing top 20 rows

root
 |-- user: string (nullable = true)
 |-- time_stamp: stri

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
# 构建结构对象
schema = StructType([
    StructField("userId", IntegerType()),
    StructField("timestamp", LongType()),
    StructField("btag", StringType()),
    StructField("cateId", IntegerType()),
    StructField("brandId", IntegerType())
])
# 从hdfs加载数据为dataframe，并设置结构
behavior_log_df = spark.read.csv("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/dataset/behavior_log.csv", header=True, schema=schema)

In [6]:
behavior_log_df.show()
behavior_log_df.count()  #   7亿

+------+----------+----+------+-------+
|userId| timestamp|btag|cateId|brandId|
+------+----------+----+------+-------+
|558157|1493741625|  pv|  6250|  91286|
|558157|1493741626|  pv|  6250|  91286|
|558157|1493741627|  pv|  6250|  91286|
|728690|1493776998|  pv| 11800|  62353|
|332634|1493809895|  pv|  1101| 365477|
|857237|1493816945|  pv|  1043| 110616|
|619381|1493774638|  pv|   385| 428950|
|467042|1493772641|  pv|  8237| 301299|
|467042|1493772644|  pv|  8237| 301299|
|991528|1493780710|  pv|  7270| 274795|
|991528|1493780712|  pv|  7270| 274795|
|991528|1493780712|  pv|  7270| 274795|
|991528|1493780712|  pv|  7270| 274795|
|991528|1493780714|  pv|  7270| 274795|
|991528|1493780765|  pv|  7270| 274795|
|991528|1493780714|  pv|  7270| 274795|
|991528|1493780765|  pv|  7270| 274795|
|991528|1493780764|  pv|  7270| 274795|
|991528|1493780633|  pv|  7270| 274795|
|991528|1493780764|  pv|  7270| 274795|
+------+----------+----+------+-------+
only showing top 20 rows



723268134

#### 分析数据集字段的类型和格式
1. 查看是否有空值
2. 查看每列数据的类型
3. 查看每列数据的类别情况

In [14]:
print("查看userId的数据情况：", behavior_log_df.groupBy("userId").count().count())
# 约113w用户
#注意：behavior_log_df.groupBy("userId").count()  返回的是一个dataframe，这里的count计算的是每一个分组的个数，但当前还没有进行计算
# 当调用df.count()时才开始进行计算，这里的count计算的是dataframe的条目数，也就是共有多少个分组

查看user的数据情况： 1136340


In [6]:
print("查看btag的数据情况：", behavior_log_df.groupBy("btag").count().collect())    # collect会把计算结果全部加载到内存，谨慎使用
# 只有四种类型数据：pv、fav、cart、buy
# 这里由于类型只有四个，所以直接使用collect，把数据全部加载出来

查看btag的数据情况： [Row(btag='buy', count=9115919), Row(btag='fav', count=9301837), Row(btag='cart', count=15946033), Row(btag='pv', count=688904345)]


In [7]:
print("查看cateId的数据情况：", behavior_log_df.groupBy("cateId").count().count())
# 约12968类别id

查看cateId的数据情况： 12968


In [8]:
print("查看brandId的数据情况：", behavior_log_df.groupBy("brandId").count().count())
# 约460561品牌id

查看brandId的数据情况： 460561


In [15]:
print("判断数据是否有空值：", behavior_log_df.count(), behavior_log_df.dropna().count())
# 约7亿条目723268134 723268134
# 本数据集无空值条目，可放心处理

判断数据是否有空值： 723268134 723268134


#### pyspark.sql.GroupedData.pivot

pivot透视操作，把某列里的字段值转换成行并进行聚合运算

如果透视的字段中的不同属性值超过10000个，则需要设置spark.sql.pivotMaxValues，否则计算过程中会出现错误。[文档介绍](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=pivot#pyspark.sql.GroupedData.pivot)。

In [7]:
# 统计每个用户对各类商品的pv、fav、cart、buy数量
cate_count_df = behavior_log_df.groupBy(behavior_log_df.userId, behavior_log_df.cateId).pivot("btag",["pv","fav","cart","buy"]).count()
cate_count_df.printSchema()    # 此时还没有开始计算

root
 |-- userId: integer (nullable = true)
 |-- cateId: integer (nullable = true)
 |-- pv: long (nullable = true)
 |-- fav: long (nullable = true)
 |-- cart: long (nullable = true)
 |-- buy: long (nullable = true)



In [18]:
cate_count_df = behavior_log_df.groupBy(behavior_log_df.userId, behavior_log_df.cateId).pivot("btag", ["pv","fav","cart","buy"]).count()
cate_count_df.show()    # 如果调用show，那么将开始执行计算，且注意这里由于pivot操作复杂，所以会比较耗时
# 113w * 1.2w

+-------+------+---+----+----+----+
| userId|cateId| pv| fav|cart| buy|
+-------+------+---+----+----+----+
| 881748| 10650|  1|null|null|null|
| 140219|  4520|  6|null|null|null|
| 418392|  4267|172|   1|null|   1|
| 570671|  6300| 38|null|null|null|
| 537700|  4292|227|null|  21|null|
| 366815|  6140|  7|null|null|null|
| 542614|  4520|178|   7|   7|   2|
| 288128|  6027|  1|null|null|null|
|1082528|  8923|  3|   1|null|null|
|1006609|  4520|  3|null|null|null|
| 409886|  6244|  4|   1|null|null|
| 209047|  4517|  1|null|null|null|
|1016137|  6092| 10|null|   1|   1|
|  33955|  6621|  5|null|null|null|
| 396941|  1478| 12|   1|null|null|
| 855789|  6428| 10|   1|   1|null|
| 391454|  4267| 11|null|null|   1|
| 509772|   311|  1|null|null|null|
|  15102|  6735|  1|null|null|null|
| 860128|  6735| 45|null|   1|   1|
+-------+------+---+----+----+----+
only showing top 20 rows



In [8]:
# 统计每个用户对各个品牌的pv、fav、cart、buy数量
brand_count_df = behavior_log_df.groupBy(behavior_log_df.userId, behavior_log_df.brandId).pivot("btag",["pv","fav","cart","buy"]).count()
# brand_count_df.show()    # 同上
# 113w * 46w

#### [dataframe.write.csv](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=write%20csv#pyspark.sql.DataFrameWriter.csv)

In [9]:
# 由于运算时间比较长，所以这里先将结果存储起来，供后续其他操作使用
# 写入数据时才开始计算
cate_count_df.write.csv("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/trained_result/preprocessing-datasets/cate_count.csv", header=True)

In [10]:
brand_count_df.write.csv("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/trained_result/preprocessing-datasets/brand_count.csv", header=True)

In [11]:
!hadoop fs -ls /workspace/3.rs_project/project1/trained_result/preprocessing-datasets

Found 2 items
drwxr-xr-x   - root supergroup          0 2019-03-21 11:21 /workspace/3.rs_project/project1/trained_result/preprocessing-datasets/brand_count.csv
drwxr-xr-x   - root supergroup          0 2019-03-21 11:22 /workspace/3.rs_project/project1/trained_result/preprocessing-datasets/cate_count.csv
